<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# install knmy
!pip install git+https://github.com/python-visualization/folium

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

In [0]:
from knmy import knmy
import pandas as pd
import numpy as np

def knmi_get(start, end, stations=[240]):
    
    # knmy.get_hourly_data returns a tuple with 4 items. Immediately index to [3] to get the df with weather variables. 
    knmi_data = knmy.get_hourly_data(stations=[240], start=start, end=end,
                            inseason=False, variables=['ALL'], parse=True)[3]
    
    # Rename columns
    cols = ['weatherstation', 'date', 'hour', 'winddirection', 'windspeed_avg', \
            'windspeed_10m','windspeed_max', 'temperature', 'temperature_min', \
            'temperature_dewpoint', 'sunduration', 'sunradiation', \
            'precipitationduration', 'precipitation', 'airpressure', \
            'horizontalview', 'cloudcover', 'relativehumidity', 'weathercode', \
            'weathercodeindicator', 'mist', 'rain', 'snow', 'storm', 'ice']
    knmi_data.columns = cols
    
    # Drop useless columns
    knmi_data.drop(['winddirection', 'windspeed_10m', 'temperature_dewpoint', \
                    'horizontalview', 'cloudcover', 'weathercode', \
                    'weathercodeindicator'], axis = 1, inplace=True)
    # Drop first row since it's actually a header, then reset index
    knmi_data.drop([0], inplace=True)
    knmi_data.reset_index(drop=True, inplace=True)
    # Subtract one hour to make data in line with NDW
    knmi_data.hour = knmi_data.hour.astype(int) - 1
    # Remove columns with NA vals
    knmi_data.dropna(axis=1, inplace=True)
    # Make dataframe numeric
    knmi_data = knmi_data.apply(pd.to_numeric)
    # Remove negative precipitation values (-1 means 0.05mm or less)
    knmi_data['precipitation'] = \
        np.where(knmi_data['precipitation'] < 0, 0, knmi_data['precipitation'])
    # Make date col a datetime object
    knmi_data['date'] = pd.to_datetime(knmi_data['date'], yearfirst = True, 
                                                  format = '%Y%m%d').dt.date
    return knmi_data